<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.883 · Aprendizaje por refuerzo</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (<i>Data science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Ejemplo: Método _TD learning_ en el entorno WindyGridWorld

En este ejemplo implementaremos el método _TD learning_ de aprendizaje por refuerzo para buscar una solución óptima en el problema de WindyGridWorld.

## 1. El entorno __WindyGridWorld__

El entorno __WindyGridWorld__ consiste en un agente que se mueve en una cuadrícula 7x10 (alto x ancho). En cada paso, el agente tiene 4 opciones de acción o movimiento: ARRIBA, ABAJO, DERECHA, IZQUIERDA. El agente siempre sale de la misma casilla [3, 0] y el juego termina cuando el agente llega a la casilla de llegada [3, 7]. 

El entorno se corresponde con el ejemplo 'Cuadrícula con viento' explicado en la sección 3.1.2. el módulo "Métodos de Diferencia Temporal". El problema radica en que hay un viento que empuja al agente hacia arriba en la parte central de la cuadrícula. Esto provoca que, aunque se ejecute una acción estándar, en la región central los estados resultantes se desplazan hacia arriba por un viento cuya fuerza varía entre columnas.

<img src="../figs/GridWorld.png">

El código para implementar este entorno, que se encuentra disponible en el fichero adjunto `windy_gridworld_env.py`, ha sido adaptado del siguiente enlace:

https://pypi.org/project/gym-gridworlds/

## 2. Métodos de Diferencia Temporal

El objetivo de este ejercicio es realizar una estimación de la política óptima mediante los métodos de Diferencia Temporal en el entorno WindyGridWorld.

A continuación, implementaremos el algoritmo de *Q-learning* explicado en el modulo "Aprendizaje por Diferencia Temporal" utilizando los siguientes parámetros:
    
- número de episodios = 200
- *learning rate* = 0.5
- *discount factor* = 1
- *epsilon* = 0.05    

Además, queremos que la solución propuesta nos permita: 

1. Mostrar por pantalla los valores Q estimados para cada estado. 
2. Mostrar por pantalla los valores de la función de valor $v_\pi(s)$ estimada para cada estado. 
3. Ejecutar un episodio siguiendo la política óptima encontrada, donde se pueda reconocer la trayectoria seguida por el agente. 

In [1]:
import matplotlib.pyplot as plt
import matplotlib.collections as mcol
from matplotlib.legend_handler import HandlerLineCollection, HandlerTuple
from matplotlib.lines import Line2D
from collections import defaultdict
import gymnasium as gym
import numpy as np
import windy_gridworld_env as wge

env = wge.WindyGridworldEnv()
print("Action space is {} ".format(env.action_space))
print("Observation space is {} ".format(env.observation_space))
print("Reward range is {} ".format(env.reward_range))

Action space is Discrete(4) 
Observation space is Tuple(Discrete(7), Discrete(10)) 
Reward range is (-inf, inf) 


In [2]:
def epsilon_greedy_policy(Q, state, nA, epsilon):
    '''
    Create a policy in which epsilon dictates how likely it will 
    take a random action.

    :param Q: links state -> action value (dictionary)
    :param state: state character is in (int)
    :param nA: number of actions (int)
    :param epsilon: chance it will take a random move (float)
    :return: probability of each action to be taken (list)
    '''
    probs = np.ones(nA) * epsilon / nA
    best_action = np.argmax(Q[state])
    probs[best_action] += 1.0 - epsilon

    return probs

In [3]:
def Q_learning(episodes, learning_rate, discount, epsilon):
    '''
    Learn to solve the environment using Q-learning

    :param episodes: Number of episodes to run (int)
    :param learning_rate: How fast it will converge to a point (float [0, 1])
    :param discount: How much future events lose their value (float [0, 1])
    :param epsilon: chance a random move is selected (float [0, 1])
    :return: x,y points to graph
    '''

    # Links state to action values
    Q = defaultdict(lambda: np.zeros(env.action_space.n))

    # Points to plot
    # number of episodes
    x = np.arange(episodes)
    # Number of steps
    y = np.zeros(episodes)
    
    for episode in range(episodes):
        state, info = env.reset()
                
        for step in range(10000):

            # Select and take action
            probs = epsilon_greedy_policy(Q, state, env.action_space.n, epsilon)
            action = np.random.choice(np.arange(len(probs)), p=probs)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
           
            # TD Update
            td_target = reward + discount * np.amax(Q[next_state])
            td_error = td_target - Q[state][action]
            Q[state][action] += learning_rate * td_error
                        
            if done:
                y[episode] = step
                break

            state = next_state   
                       
    return x, y, Q

In [4]:
x, y, q = Q_learning(episodes=200, learning_rate=0.5, discount=1, epsilon=0.05)

In [5]:
#mostrar los Q de cada estado-acción y la función valor

f = open('output.txt', 'w')
for i in range(7):
    for j in range(10):
        V = max(q[i,j])    
        print((i,j), q[i,j], V)
        print((i,j), q[i,j], V, file=f)

(0, 0) [-13.75       -13.85735168 -14.00598127 -13.90147955] -13.75
(0, 1) [-13.97783425 -13.73118302 -13.68080919 -13.54325715] -13.543257146388758
(0, 2) [-13.34367536 -12.96898183 -13.15315163 -13.12029874] -12.968981833022852
(0, 3) [-12.         -11.99975215 -12.30550682 -12.78366594] -11.999752151664612
(0, 4) [-11.         -11.         -11.48477277 -12.36204311] -10.99999999985788
(0, 5) [-10.0511322  -10.         -10.49999606 -10.1404083 ] -10.0
(0, 6) [ -9.75        -9.          -9.73026627 -10.03380357] -9.0
(0, 7) [-8.94721942 -8.         -8.75621012 -9.95173485] -8.0
(0, 8) [-7.75       -7.         -7.58955466 -8.80324484] -7.0
(0, 9) [-6.66445381 -6.87499836 -6.         -7.89870333] -6.0
(1, 0) [-14.05913377 -14.14331057 -13.94401342 -13.93417358] -13.934173583984375
(1, 1) [-14.06056597 -13.71476294 -13.84079686 -14.02725999] -13.714762935828382
(1, 2) [-13.23369961 -12.95218869 -13.09661504 -13.76533725] -12.952188691231367
(1, 3) [-12.44179693 -11.99959376 -12.         

In [6]:
# función que muestra los valores de la función de estado V(s) en la cuadricula

def print_values(Q, height, width):
    for i in range(height):
        print("------------------------------------------------------------------------------------------")
        for j in range(width):
            arr = np.array(Q[i,j])
            v = np.amax(arr)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="") # -ve sign takes up an extra space
        print("")

In [7]:
print_values(q, 7, 10)

------------------------------------------------------------------------------------------
-13.75|-13.54|-12.97|-12.00|-11.00|-10.00|-9.00|-8.00|-7.00|-6.00|
------------------------------------------------------------------------------------------
-13.93|-13.71|-12.95|-12.00|-11.00|-10.00|-9.00|-7.95|-6.84|-5.00|
------------------------------------------------------------------------------------------
-14.53|-13.95|-12.98|-12.00|-11.00|-10.00|-9.00|-5.00|-5.88|-4.00|
------------------------------------------------------------------------------------------
-15.00|-14.00|-13.00|-12.00|-11.00|-9.91|-8.73| 0.00|-4.00|-3.00|
------------------------------------------------------------------------------------------
-14.33|-13.56|-12.94|-11.99|-10.75|-9.65| 0.00|-0.75|-1.00|-2.00|
------------------------------------------------------------------------------------------
-13.69|-13.11|-12.36|-11.50|-10.34| 0.00| 0.00| 0.00|-1.00|-1.86|
-------------------------------------------------------

In [8]:
# ejecución de un episodio siguiendo la politica optima

def execute_episode_TD(q, env):
    obs, info = env.reset()
    t, total_reward, done = 0, 0, False

    print("Obs inicial: {} ".format(obs))

    switch_action = {
            0: "U",
            1: "R",
            2: "D",
            3: "L",
        }

    for t in range(1000): # limitamos el número de time-steps de cada episodio a 1000
        
        # Elegir una acción siguiendo la política óptima
        arr = np.array(q[obs])
        action = arr.argmax()
       
        # Ejecutar la acción y esperar la respuesta del entorno
        new_obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        obs = new_obs
        print("Action: {} -> Obs: {} and reward: {}".format(switch_action[action], obs, reward))

        if t==999:
            print("Number of time-septs exceeds 1000. STOP episode.") 
        total_reward += reward
        t += 1
        if done:
            break
   
    print("Episode finished after {} timesteps and reward was {} ".format(t, total_reward))
    env.close()

In [9]:
execute_episode_TD(q, env)

Obs inicial: (3, 0) 
Action: R -> Obs: (3, 1) and reward: -1
Action: R -> Obs: (3, 2) and reward: -1
Action: R -> Obs: (3, 3) and reward: -1
Action: R -> Obs: (2, 4) and reward: -1
Action: R -> Obs: (1, 5) and reward: -1
Action: R -> Obs: (0, 6) and reward: -1
Action: R -> Obs: (0, 7) and reward: -1
Action: R -> Obs: (0, 8) and reward: -1
Action: R -> Obs: (0, 9) and reward: -1
Action: D -> Obs: (1, 9) and reward: -1
Action: D -> Obs: (2, 9) and reward: -1
Action: D -> Obs: (3, 9) and reward: -1
Action: D -> Obs: (4, 9) and reward: -1
Action: L -> Obs: (4, 8) and reward: -1
Action: L -> Obs: (3, 7) and reward: -1
Episode finished after 15 timesteps and reward was -15 
